In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("dirty_financial_transactions.csv")

In [3]:
df.head()

,Transaction_ID,Transaction_Date,Customer_ID,Product_Name,Quantity,Price,Payment_Method,Transaction_Status
0,T0001,2024-08-02,C2205,Headphones,-5.0,$420.21,pay pal,NaN
1,T0002,2020-02-10,C3156,Coffee,469.0,-445.34202525395585,creditcard,Pending
2,T0003,2025-02-30,C2919,Tablet,-4.0,810.9930123946459,credit card,completed
3,T0004,2020-08-17,C3009,Tab,-7.0,868.6083413217348,PayPal,Pending
4,T0005,2025-02-30,C3488,Coffee Machine,-10.0,-763.1224490039416,PayPal,completed


## EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Transaction_ID      94982 non-null   object 
 1   Transaction_Date    95120 non-null   object 
 2   Customer_ID         95122 non-null   object 
 3   Product_Name        100000 non-null  object 
 4   Quantity            94981 non-null   float64
 5   Price               66503 non-null   object 
 6   Payment_Method      100000 non-null  object 
 7   Transaction_Status  83321 non-null   object 
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


In [5]:
df.isnull().sum() 

Transaction_ID         5018
Transaction_Date       4880
Customer_ID            4878
Product_Name              0
Quantity               5019
Price                 33497
Payment_Method            0
Transaction_Status    16679
dtype: int64

In [6]:
df.nunique()

Transaction_ID        94040
Transaction_Date       1861
Customer_ID            5000
Product_Name             46
Quantity                921
Price                 65757
Payment_Method            7
Transaction_Status        5
dtype: int64

## Data cleaning

In [7]:
df = df.drop_duplicates() # drop duplicate rows

### Cleaning Transaction IDs

In [8]:
# clean Transaction_ID by stripping spaces
df['Transaction_ID'] = df['Transaction_ID'].astype(str).str.strip()

# filter to keep only valid Transaction_IDs (e.g., T followed by digits)
df = df[df['Transaction_ID'].str.match(r'^T\d+$', na=False)]

# extract numeric part for sorting
df['Txn_Num'] = df['Transaction_ID'].str.replace('T', '', regex=False).astype(int)

# sort by Txn_Num 
df = df.sort_values('Txn_Num')

# rearrange columns
cols = ['Transaction_ID', 'Transaction_Date', 'Customer_ID', 'Product_Name',
        'Quantity', 'Price', 'Payment_Method', 'Transaction_Status']

df = df[cols]

### Standardise date formats

In [9]:
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], format='%Y-%m-%d', errors='coerce')

Some Transaction_Date entries were invalid dates such as 30th of February and they have since been replace with "NaT".

### Standardise product names

In [10]:
df["Product_Name"].unique()

product_standardisation_map = {
    # coffee related
    'Coffee': 'Coffee Machine',
    'Coffee ': 'Coffee Machine',
    'Cof': 'Coffee Machine',
    'Coff': 'Coffee Machine',
    'Coffe': 'Coffee Machine',
    'Co': 'Coffee Machine',
    'C': 'Coffee Machine',
    'Coffee M': 'Coffee Machine',
    'Coffee Ma': 'Coffee Machine',
    'Coffee Mac': 'Coffee Machine',
    'Coffee Mach': 'Coffee Machine',
    'Coffee Machi': 'Coffee Machine',
    'Coffee Machin': 'Coffee Machine',
    'Coffee Machine': 'Coffee Machine',

    # headphones
    'Headphones': 'Headphones',
    'Headp': 'Headphones',
    'Head': 'Headphones',
    'Hea': 'Headphones',
    'He': 'Headphones',
    'H': 'Headphones',
    'Headphon': 'Headphones',
    'Headphone': 'Headphones',
    'Headph': 'Headphones',
    'Headpho': 'Headphones',

    # laptop
    'Laptop': 'Laptop',
    'Lapt': 'Laptop',
    'Lap': 'Laptop',
    'Lapto': 'Laptop',
    'L': 'Laptop',
    'La': 'Laptop',

    # smartphone
    'Smartphone': 'Smartphone',
    'Smartph': 'Smartphone',
    'Smartpho': 'Smartphone',
    'Smartphon': 'Smartphone',
    'Smartp': 'Smartphone',
    'Smar': 'Smartphone',
    'Sma': 'Smartphone',
    'Sm': 'Smartphone',
    'S': 'Smartphone',
    'Smart': 'Smartphone',

    # tablet
    'Tablet': 'Tablet',
    'Tab': 'Tablet',
    'Tabl': 'Tablet',
    'Table': 'Tablet',
    'Ta': 'Tablet',
    'T': 'Tablet',
}

df['Product_Name'] = df['Product_Name'].str.strip()  # remove extra spaces
df['Product_Name'] = df['Product_Name'].map(product_standardisation_map).fillna(df['Product_Name'])  # apply mapping

### Transforming negative quantity values to positive

In [11]:
df['Quantity'] = df['Quantity'].abs()

### Remove quantity outliers 

In [12]:
df.boxplot()

q1 = df['Quantity'].quantile(0.25)
q3 = df['Quantity'].quantile(0.75)
iqr = q3 - q1

# define upper limit as Q3 + 1.5*IQR
upper_limit = q3 + 1.5 * iqr

# keep values that are below the defined upper limit
df = df[df['Quantity'] <= upper_limit]

### Standardise price formats and convert negative values to positive

In [15]:
# remove '$' and convert to numeric
df['Price'] = pd.to_numeric(df['Price'].astype(str).str.replace('$', '', regex=False), errors='coerce')

# convert to positive
df['Price'] = df['Price'].abs()

# round to 2 decimal places
df['Price'] = df['Price'].round(2)

### Standardise payment method

In [16]:
df['Payment_Method'].unique()

payment_standardisation_map = {
    # paypal
    'pay pal' : 'PayPal',
    
    # credit card
    'creditcard' : 'Credit Card',
    'credit card' : 'Credit Card'
}

df['Payment_Method'] = df['Payment_Method'].str.strip()
df['Payment_Method'] = df['Payment_Method'].map(payment_standardisation_map).fillna(df['Payment_Method'])

### Standardise transaction status

In [17]:
df['Transaction_Status'].unique()

transaction_standardisation_map = {
    # Completed
    'completed' : 'Completed',
    'complete' : 'Completed'
}

df['Transaction_Status'] = df['Transaction_Status'].str.strip()
df['Transaction_Status'] = df['Transaction_Status'].map(transaction_standardisation_map).fillna(df['Transaction_Status'])

## Output to csv

In [25]:
df.to_csv('cleaned_financial_transactions.csv', index=False)